In [1]:
# import the optimize solver Gurobi and pandas
from gurobipy import * 

In [2]:
number_of_product = 30 # Set the index i for number of product
number_of_shelf = 14 # Thare are total 14 shelves
m = Model() # Import and create the model


--------------------------------------------
--------------------------------------------

Using license file C:\Users\chaoz\gurobi_licence\gurobi.lic
Academic license - for non-commercial use only


In [3]:
# Set the input Parameter: 
units = [23, 15, 12, 18, 36, 50, 38, 18,32, 30, 42, 31, 11, 25, 57, 48, 32, 32, 39, 27, 5, 29, 14, 38, 34, 19, 6, 55, 52, 2] # units of product i
price = [53000, 31000, 25000,40000,75000,103000,82000,40000,69000,66000,93000,70000,26000,55000,121000,104000,72000,73000,81000,63000,13000,60000,36000,79000,68000,41000,17000,111000,113000,12000] # Price of product i
max_unit = 60  # Maximum units of product in a shelf

In [4]:
# Set the Variable list: Shelf number where product i is kept
# Set the variable to binary
nx = []
for i in range(number_of_product):
    k = []
    for j in range(number_of_shelf):
        k.append(m.addVar(vtype=GRB.BINARY, name='nx{}'.format(str(i + 1) +'&' +str(j + 1))))
    nx.append(k)

In [5]:
# Caculate the units stocked and price total for each shelf
units_stocked = []
for j in range(number_of_shelf):
    units_stocked.append(quicksum([nx[i][j] * units[i] for i in range(number_of_product)]))

price_total = []
for j in range(number_of_shelf):
    price_total.append(quicksum([nx[i][j] * price[i] for i in range(number_of_product)]))
    

In [6]:
# Set the Maximize Objective: Total price
m.setObjective(sum([price_total[j] for j in range(number_of_shelf)]) ,  GRB.MAXIMIZE)

In [7]:
# Meet the constriant that the shelves can hold 60 units of products each
m.addConstrs((units_stocked[j] <= 60 for j in range(14)), name='cx')
m.update()

In [8]:
# Run the optimize solver
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 14 rows, 420 columns and 420 nonzeros
Model fingerprint: 0xc0e1be44
Variable types: 0 continuous, 420 integer (420 binary)
Coefficient statistics:
  Matrix range     [2e+00, 6e+01]
  Objective range  [1e+04, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+01, 6e+01]
Found heuristic solution: objective 1876000.0000
Presolve removed 13 rows and 398 columns
Presolve time: 0.18s
Presolved: 1 rows, 22 columns, 22 nonzeros
Found heuristic solution: objective 2147000.0000
Variable types: 0 continuous, 22 integer (21 binary)

Root relaxation: objective 2.157333e+06, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2157333.33    0    1 2147000.00 2157333.33  0.48%     -    0s
H    0     0                    2156000.0000 2157333.33  0.06%     -    0s
     0     0 

In [9]:
# Get the Optimal Solution for X
import pandas as pd
dics = {}
for i in range(14):
    dics[i] = m.X[i*30 :(i+1)*30]
    
xxxx = pd.DataFrame(dics).replace(-0.0, 0).astype('int')

In [10]:
# Get the Optimal Objective Value
m.ObjVal

2156000.0

In [11]:
for j in range(number_of_shelf):
    print(sum([xxxx.iloc[i,j] * units[i] for i in range(30)]))

0
0
0
0
0
320
38
0
115
272
440
0
482
384
